In [23]:
# Import Libraries
import os
from getpass import getpass
from urllib.request import urlopen

# All langchain-based
from langchain_elasticsearch import ElasticsearchStore
from langchain.vectorstores import ElasticVectorSearch
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings.openai import OpenAIEmbeddings

# Data Manipulation 
import pandas as pd

# All Elasticsearch based
from elasticsearch import Elasticsearch

In [ ]:
## IDs and Passwords
ELASTIC_CLOUD_ID = "ab9fc665e06642b4b2c99c5f9898e875:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDg3MzIwMzg0YWMzNzRjNTJiYmQ3YWFkYjI4NWUxNjExJDIxMDFiZTgzNjg5ODQ0MmFhMzg4ZTMzYzU5MTdmOTlh"
ELASTIC_API_KEY = "WUoyVkpKTUJ0ZTlkVlhESWtSVDQ6eGUwY1dzcmlSbGE4bW5iY2gxQl9CZw=="
ELASTIC_PASSWORD = "GjcdSvHrx8Mh1Qy7ZB93m3DX"
OPENAI_API_KEY = "sk-JDd8XASldxnf4JlfzqcFX6IZkRp88UQSOuS6MqsBgLT3BlbkFJoaFWr04LuZoWZQlTBdTzxzbQ7tgFTD2iXbICVfLxcA"


In [4]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key :")

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/var/folders/m0/nhpw880n2pbdfh3k943xczbr0000gn/T/ipykernel_84327/1020807959.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [6]:
# Initialize Elasticsearch client
es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

# Test the connection
if es.ping():
    print("Connected to ElasticCloud!")
else:
    print("Connection failed. Check your credentials.")

Connected to ElasticCloud!


In [8]:
# Test the Elasticsearch connection
if es.ping():
    print("Connected to ElasticCloud!")
else:
    print("Connection failed. Check your credentials.")

Connected to ElasticCloud!


In [9]:
# A new index name
elastic_index_name = "openai_mit_llama_index"

In [15]:
if es.indices.exists(index=elastic_index_name):
    res = es.indices.delete(index=elastic_index_name)
    print(f"Deleted index '{elastic_index_name}': {res}")
else:
    print(f"Index '{elastic_index_name}' does not exist.")

Index 'openai_mit_llama_index' does not exist.


In [16]:
# Check if the index already exists
if not es.indices.exists(index=elastic_index_name):
    es.indices.create(index=elastic_index_name)
    print(f"Index '{elastic_index_name}' created successfully.")
else:
    print(f"Index '{elastic_index_name}' already exists.")

Index 'openai_mit_llama_index' created successfully.


In [ ]:
news_articles = pd.read_csv('/content/drive/MyDrive/kaggle/data/bbc_news_articles.csv', index_col=None)

In [ ]:
news_articles.head()

,Unnamed: 0,Published Date,Author,Source,Article Header,Sub_Headings,Article Body,Url
0,0,"July 7, 2023",Adam Zewe,MIT News Office,Learning the language of molecules to predict ...,This AI system only needs a small amount of da...,['Discovering new materials and drugs typicall...,https://news.mit.edu/2023/learning-language-mo...
1,1,"July 6, 2023",Alex Ouyang,Abdul Latif Jameel Clinic for Machine Learning...,MIT scientists build a system that can generat...,"BioAutoMATED, an open-source, automated machin...",['Is it possible to build machine-learning mod...,https://news.mit.edu/2023/bioautomated-open-so...
2,2,"June 30, 2023",Jennifer Michalowski,McGovern Institute for Brain Research,"When computer vision works more like a brain, ...",Training artificial neural networks with data ...,"['From cameras to self-driving cars, many of t...",https://news.mit.edu/2023/when-computer-vision...
3,3,"June 30, 2023",Mary Beth Gallagher,School of Engineering,Educating national security leaders on artific...,"Experts from MIT’s School of Engineering, Schw...",['Understanding artificial intelligence and ho...,https://news.mit.edu/2023/educating-national-s...
4,4,"June 30, 2023",Adam Zewe,MIT News Office,Researchers teach an AI to write better chart ...,A new dataset can help scientists develop auto...,['Chart captions that explain complex trends a...,https://news.mit.edu/2023/researchers-chart-ca...


In [ ]:
news_articles["Article Body"]

,Article Body
0,['Discovering new materials and drugs typicall...
1,['Is it possible to build machine-learning mod...
2,"['From cameras to self-driving cars, many of t..."
3,['Understanding artificial intelligence and ho...
4,['Chart captions that explain complex trends a...
...,...
1013,"[""CAMBRIDGE, Mass.--Consider the fish: highly ..."
1014,"['Cog, the newest and most ambitious robot dev..."
1015,"['A few years from now, patients with polyps o..."
1016,"[""Despite a wind chill index of -80oF and an e..."


In [ ]:
DOCUMENT = news_articles["Article Body"]
HEADING = news_articles["Article Header"]

In [ ]:
type(news_articles["Article Body"][0])

str

In [ ]:
# Prepare metadata and content
metadata = []
content = []

for _, row in news_articles.iterrows():
    content.append(row["Article Body"])  # Store article body as content
    metadata.append({"heading": row["Article Header"]})  # Store heading as metadata

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.create_documents(content, metadatas=metadata)

In [ ]:
# metadata = []
# content = []

# for _, row in news_articles.iterrows():
#   content.append(row["Article Header"])
#   metadata.append(
#       {
#           "heading": row["Article Body"],
#       }
#   )

# text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0)
# docs = text_splitter.create_documents(content, metadatas=metadata)

In [ ]:
# len(docs)

1018

In [14]:
# from sentence_transformers import SentenceTransformer

# # Load a sentence-transformer model
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# query_embedding = GoogleGenerativeAIEmbeddings(
#     model="models/embedding-001", task_type="retrieval_document"
# )

# es = ElasticsearchStore.from_documents(
#     docs,
#     es_cloud_id=ELASTIC_CLOUD_ID,
#     es_api_key=ELASTIC_API_KEY,
#     index_name=elastic_index_name,
#     embedding=query_embedding,
# )

In [ ]:
# Load the latest docs into the database
# documents = vector_store.from_documents(
#     docs,
#     embeddings,
#     index_name=elastic_index_name,
#     es_cloud_id=ELASTIC_CLOUD_ID,
#     es_api_key=ELASTIC_API_KEY,
# )

In [ ]:
## Batch Processing

In [ ]:
# Okay I might need to introduce some changes over here later AND so ON and so forth

In [16]:
# batch_size = 50
# for i in tqdm(range(0, len(docs), batch_size)):
#   batch = docs_subset[i:i+batch_size]
#   vector_store.from_documents(batch,
#                         embeddings,
#                         index_name=elastic_index_name,
#                         es_cloud_id=ELASTIC_CLOUD_ID,
#                         es_api_key=ELASTIC_API_KEY
#   )

In [ ]:
# import time
# from openai.error import RateLimitError

In [ ]:
# import tiktoken

# tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002")
# tokens = sum(len(tokenizer.encode(doc.page_content)) for doc in docs)
# print(f"Total tokens: {tokens}")

Total tokens: 1057214


In [ ]:
# Retry up to 5 times
# for attempt in range(5):
#   try:
#     documents = vector_store.from_documents(
#                         docs,
#                         embeddings,
#                         index_name=elastic_index_name,
#                         es_cloud_id=ELASTIC_CLOUD_ID,
#                         es_api_key=ELASTIC_API_KEY,
#                 )
#     break
#   except RateLimitError:
#     wait_time = (2 ** attempt) * 4  # Exponential backoff
#     print(f"Rate limit hit. Retrying in {wait_time} seconds...")
#     time.sleep(wait_time)

In [11]:
# from langchain.schema import BaseRetriever, Document
# from typing import List

# class HeadingBasedRetriever(BaseRetriever):
#   def __init__(self, vector_store, top_k: int = 1):
#     self.vector_store = vector_store
#     self.top_k = top_k

#   def _get_relevant_documents(self, query: str) -> List[Document]:
#     # Retrieve top-k documents based on heading similarity
#     results = self.vector_store.similarity_search(
#         query=query,
#         k=self.top_k,
#         filter=None  # Optional: Add filters if needed
#     )

#     # Extract and return the article body of the top result
#     if results:
#       top_result = results[0]
#       return [Document(page_content=top_result.page_content, metadata=top_result.metadata)]

#     return []  # Return an empty list if no results are found

**Change this later. But first pf all, even if something otherwise is being returned, do it**

In [17]:
from typing import List, Any
from langchain.schema import BaseRetriever, Document
from pydantic import Field

class HeadingBasedRetriever(BaseRetriever):
  vector_store: Any  # Define vector_store as a field (type can be adjusted based on your implementation)
  top_k: int = Field(default=1)  # Define top_k with a default value

  def _get_relevant_documents(self, query: str) -> List[Document]:
    # Retrieve top-k documents based on heading similarity
    results = self.vector_store.similarity_search(
        query=query,
        k=self.top_k,
        filter=None  # Optional: Add filters if needed
    )

    # Extract and return the article body of the top result
    if results:
        top_result = results[0]
        return [Document(page_content=top_result.page_content, metadata=top_result.metadata)]

    return []  # Return an empty list if no results are found

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain

# Initialize retriever
retriever = HeadingBasedRetriever(vector_store=vector_store)

In [25]:
# Step 2: Check if the index exists
if es.indices.exists(index=elastic_index_name):
    print(f"Index '{elastic_index_name}' exists.")

    # Step 3: Fetch and print the first few documents
    response = es.search(index=elastic_index_name, query={"match_all": {}}, size=10)
    print("Documents in the index:")
    for hit in response['hits']['hits']:
        print(f"ID: {hit['_id']}, Source: {hit['_source']["metadata"]}")
else:
    print(f"Index '{elastic_index_name}' does not exist.")

Index 'openai_mit_llama_index' exists.
Documents in the index:
ID: 45e59948-1994-456d-b310-4c0306bcb722, Source: {'date': '2023-06-30T00:00:00', 'author': 'Jennifer Michalowski', 'source': 'McGovern Institute for Brain Research', 'heading': 'When computer vision works more like a brain, it sees more like people do', 'description': 'Training artificial neural networks with data from real brains can make computer vision more robust.'}
ID: 8f112e9c-206c-4398-ab4f-0dae1d4a56ea, Source: {'date': '2023-06-22T00:00:00', 'author': 'Mary Beth Gallagher', 'source': 'School of Engineering', 'heading': 'MIT-Pillar AI Collective announces first seed grant recipients', 'description': 'Six teams conducting research in AI, data science, and machine learning receive funding for projects that have potential commercial applications.  '}
ID: a36a7bc1-8eb8-4ebb-8d70-c9a83caea9f9, Source: {'date': '2023-06-16T00:00:00', 'author': 'Terri Park', 'source': 'MIT Schwarzman College of Computing', 'heading': 'Env

In [20]:
context = retriever._get_relevant_documents("What does the article talk about J-WAFS Grand Challenge?")

In [21]:
context

[]

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = OpenAI()

In [ ]:
llm_chat = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
# Change mode of instruction, better prompting, better settings, so ON and so forth. All those changes towards the end. For now, just focus on development of the business prototype

In [ ]:
query = "What does the article talk about J-WAFS Grand Challenge?"

In [ ]:
# Retrieve context and invoke chain
# retrieved_docs = retriever._get_relevant_documents(query)
# if retrieved_docs:
#   context = retrieved_docs[0].page_content  # Use top-1 document's content
# else:
#   context = "No relevant documents found."

In [ ]:
# Add String Output Parser
parser = StrOutputParser()

In [ ]:
# Define prompt template
ANSWER_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for answering questions. Use the following "Article Content" to answer the question.

    Article Content: {context}
    Question: "{question}"
    Answer:
    """
)

# Define chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | ANSWER_PROMPT
    | llm_chat # Replace with your LLM instance (e.g., OpenAI or similar)
    | parser
)

# Query example
response = chain.invoke(query)

# Split response into lines (e.g., after every sentence)
formatted_response = "\n".join(response.split(". "))  # Add line breaks after sentences

print("---- Answer ----")
print(formatted_response)

---- Answer ----
The article discusses the inaugural J-WAFS Grand Challenge, which aims to develop enhanced crop variants and transition them from the laboratory to agricultural fields
Established by the Abdul Latif Jameel Water and Food Systems Lab (J-WAFS) at MIT, the Grand Challenge seeks to inspire researchers to work towards a water-secure and food-secure future in response to global challenges such as climate change and food insecurity.

The challenge awarded a $1.5 million grant to a team led by Professor Matt Shoulders and research scientist Robert Wilson to address inefficiencies in photosynthesis, particularly focusing on improving the RuBisCO enzyme crucial for carbon fixation in plants
The project will employ advanced synthetic and computational biology techniques to engineer better RuBisCO variants, which could significantly enhance crop yields.

The article highlights the importance of food systems in the context of climate change and greenhouse gas emissions, and it emph

## Google AI Studio Approach


In [ ]:
# import google.generativeai as genai

# genai.configure(api_key=GOOGLE_API_KEY)
# llm_model = genai.GenerativeModel("gemini-1.5-flash")
# response = llm.generate_content("Explain how AI works")
# print(response.text)

In [ ]:
# !curl -X POST "http://127.0.0.1/v1beta/models/gemini-1.5-flash:generateContent" \
# -H "Content-Type: application/json" \
# -d '{"prompt": "Hello, world!", "max_tokens": 100}' -v

In [ ]:
# Define the mapping
# mappings = {
#     "properties": {
#         "content_vector": {
#             "type": "dense_vector",
#             "dims": 384,
#             "index": "true",
#             "similarity": "cosine",
#         }
#     }
# }

# # Check if the index exists
# if es.indices.exists(index=elastic_index_name):
#     # Delete the existing index
#     es.indices.delete(index=elastic_index_name)
#     print(f"Deleted existing index: {elastic_index_name}")

# # Create a new index with the mapping
# es.indices.create(index=elastic_index_name, mappings=mappings)
# print(f"Created new index: {elastic_index_name}")

In [ ]:
# import google.generativeai as genai
# import numpy as np

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

# from google.api_core import retry

In [ ]:
# def make_embed_text_fn(model):
#   @retry.Retry(timeout=300.0)
#   def embed_fn(text: str) -> list[float]:
#     # Set the task_type to CLUSTERING.
#     embedding = genai.embed_content(model=model,
#                                     content=text,
#                                     task_type="retrieval_document")
#     return np.array(embedding)

#   return embed_fn

# def create_embeddings(df):
#   model = 'models/embedding-001'
#   df["Article Body Embeddings"] = df["Article Header"].progress_apply(make_embed_text_fn(model))
#   return df

In [ ]:
# Generate embeddings for each chunk
# news_articles['embeddings'] = news_articles["Article Header"].progress_apply(lambda doc: embedding_model.encode(doc))

# Check sample embeddings
# print(news_articles['embeddings'].head())

  0%|          | 0/1018 [00:00<?, ?it/s]

0    [-0.030335475, -0.11050753, 0.0033278002, 0.00...
1    [-0.09683153, -0.03105063, 0.0058410014, -0.00...
2    [0.0521426, -0.07336696, 0.015279335, -0.05729...
3    [0.0010559065, 0.004276596, 0.018455416, 0.028...
4    [0.017620977, -0.017530331, 0.034623932, 0.018...
Name: embeddings, dtype: object


In [ ]:
## news_articles_sub = news_articles[["Article Header", "Article Body", "embeddings"]]

In [ ]:
# news_articles_sub = create_embeddings(news_articles_sub)

In [ ]:
# news_articles_sub["Embedding"] = news_articles_sub["Article Body Embeddings"].apply(lambda row: row["embedding"])

In [ ]:
# type(news_articles_sub["Embedding"][0])

In [ ]:
## news_collection = []

In [ ]:
# for i, row in tqdm(news_articles_sub.iterrows()):
#   news_collection.append({"index": {"_index": elastic_index_name}})
#   news_data = {}
#   news_data["heading"] = row["Article Header"]
#   news_data["content"] = row["Article Body"]

#   # Transforming the title into an embedding using the model
#   news_data["content_vector"] = row["embeddings"]
#   news_collection.append(news_data)

0it [00:00, ?it/s]

In [ ]:
# Save to CSV
# news_articles_sub.to_csv("/content/drive/MyDrive/kaggle/data/news_articles_sub.csv", index=False)

In [ ]:
# Do bulk import into the database
# es.bulk(index=elastic_index_name, operations=news_collection, refresh=True)

In [ ]:
# from langchain.schema import BaseRetriever, Document
# from typing import List

# class HeadingBasedRetriever(BaseRetriever):
#     def __init__(self, vector_store, top_k: int = 1):
#         self.vector_store = vector_store
#         self.top_k = top_k

#     def _get_relevant_documents(self, query: str) -> List[Document]:
#         # Retrieve top-k documents based on heading similarity
#         results = self.vector_store.similarity_search(
#             query=query,
#             k=self.top_k,
#             filter=None  # Optional: Add filters if needed
#         )

#         # Extract and return the article body of the top result
#         if results:
#             top_result = results[0]
#             return [Document(page_content=top_result.page_content, metadata=top_result.metadata)]

#         return []  # Return an empty list if no results are found

In [ ]:
def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
      print("Your search returned no results.")
    else:
      for hit in response["hits"]["hits"]:
        id = hit["_id"]
        heading = hit["_source"]["heading"]
        content = hit["_source"]["content"]
        pretty_output = f"\nID: {id}\nHeading: {heading}\nContent: {content}"
        print(pretty_output)

In [ ]:
# query = "What does the article talk about J-WAFS Grand Challenge"

In [ ]:
# def make_embed_text(model, text):
#   @retry.Retry(timeout=300.0)
#   def embed_fn(text: str):
#     # Set the task_type to CLUSTERING.
#     embedding = genai.embed_content(model=model,
#                                     content=text,
#                                     task_type="retrieval_document")["embedding"]
#     return embedding

#   return embed_fn

In [ ]:
# def ma                                    ke_embedding(text):
#   return genai.embed_content(model=model,
#                                     content=query,
# task_type="retrieval_document")["embedding"]

In [ ]:
# response = es.search(
#     index=elastic_index_name,
#     knn={
#         "field": "content_vector",
#         "query_vector": embedding_model.encode(query),
#         "k": 10,
#         "num_candidates": 100,
#     },
#     size=100
# )

# pretty_response(response)

In [ ]:
# documents = [hit["_source"]["content"] for hit in response["hits"]["hits"]]
# documents

In [ ]:
# Join the documents into a single context string
# context = " ".join(documents)

# Truncate the context if it exceeds the token limit
# max_tokens = 10000  # Adjust based on your LLM's capabilities
# context = context[:max_tokens]

In [ ]:
# prompt_template = f"""
# Relevant context: {context}

# Considering the relevant context, answer the question.

# Question: {query}

# Answer:
# """

In [ ]:
# prompt_template

In [ ]:
# def ask_llama(prompt_template, max_length=300):
#   response = llama_pipeline(prompt_template, num_return_sequences=1)
#   return response[0]["generated_text"]

In [ ]:
# messages = [
# 	{
# 		"role": "user",
# 		"content": prompt_template
# 	}
# ]

# stream = client.chat.completions.create(
#   model="meta-llama/Meta-Llama-3-8B-Instruct",
# 	messages=messages,
# 	max_tokens=1000,
# 	stream=True
# )

# for chunk in stream:
#     print(chunk.choices[0].delta.content, end="")

According to the relevant context, the article talks about the J-WAFS Grand Challenge Grant, a research program established by the Abdul Latif Jameel Water and Food Systems Lab (J-WAFS) at MIT. The program aims to inspire MIT researchers to work towards a water-secure and food-secure future by tackling complex challenges related to agriculture, water, and food systems. Specifically, the article highlights the inaugural J-WAFS Grand Challenge Grant, which has been awarded to a team of researchers led by Professor Matt Shoulders and research scientist Robert Wilson to improve the efficiency of photosynthesis in crops.

In [ ]:
# model_name="meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
# Initialize conversation history
# messages = []

In [ ]:
# def multi_turn_conversation(prompt, max_tokens=10000):
#     """
#     Handles a single turn in a multi-turn conversation.

#     Args:
#         query (str): User's query.
#         client: Client object for interacting with the model API.
#         model_name (str): The name of the model to use.
#         max_tokens (int): Maximum tokens for the model's response.

#     Returns:
#         str: The assistant's response.
#     """
#     global messages  # Access the global conversation history

#     # Append the user's query to the conversation history
#     messages.append({"role": "user", "content": prompt})

#     # Send the conversation history to the model
#     stream = client.chat.completions.create(
#         model=model_name,
#         messages=messages,
#         max_tokens=max_tokens,
#         stream=True
#     )

#     # Collect the streamed response
#     response = ""
#     for chunk in stream:
#         content = chunk.choices[0].delta.content
#         print(content, end="")  # Stream the response in real-time
#         response += content

#     print("\n")  # Add a newline after streaming the response

#     # Append the assistant's response to the conversation history
#     messages.append({"role": "assistant", "content": response})

#     return response

In [ ]:
# Start the conversation
# print("Assistant: Hello! How can I assist you today?\n")

# # Example conversation loop
# while True:
#     user_query = input("You: ")  # Take input from the user
#     if user_query.lower() in ["exit", "quit"]:
#         print("Assistant: Goodbye!")
#         break

#     # Pass the query to the multi-turn function
#     prompt_template = f"""
#     Relevant context: {context}

#     Considering the relevant context, answer the question.

#     Question: {user_query}

#     Answer:
#     """
#     assistant_response = multi_turn_conversation(prompt_template)

In [ ]:
# response = ask_llama(prompt_template)
# print(response.text)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import LLMChain

# Initialize retriever
# retriever = HeadingBasedRetriever(vector_store)

In [ ]:
# # Define prompt template
# ANSWER_PROMPT = ChatPromptTemplate.from_template(
#     """You are an assistant for answering questions. Use the following article content to answer the question.

#     Article Content: {context}
#     Question: "{question}"
#     Answer:
#     """
# )

# # Define chain
# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | ANSWER_PROMPT
#     | llm  # Replace with your LLM instance (e.g., OpenAI or similar)
# )

# # Query example
# query = "What is NASA's sales team?"
# answer = chain.invoke(query)

# print("---- Answer ----")
# print(answer)

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain, ConversationChain

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [59]:
## Example
prompt_template_name = PromptTemplate(
    input_variables=["context", "question"],
    template = """You are an assistant for answering questions. Use the following article content to answer the question.

    Article Content: {context}
    Question: {question}
    Answer:
    """)

In [60]:
chain = LLMChain(llm=llm_chat, prompt=prompt_template_name)

In [61]:
print(chain.run({"context": context, "question": query}))

The article discusses the J-WAFS Grand Challenge, which is an initiative established by the Abdul Latif Jameel Water and Food Systems Lab (J-WAFS) at MIT to inspire researchers to work towards achieving a water-secure and food-secure future. Launched in 2022, this grant program encourages innovative projects that address the interrelated challenges of climate, water, and food. The inaugural grant has been awarded to a research team led by Professor Matt Shoulders, focusing on improving the efficiency of the photosynthetic enzyme RuBisCO. The project, named the Enhanced Photosynthesis in Crops (EPiC) platform, aims to develop better RuBisCO variants through advanced protein engineering techniques, potentially leading to significant improvements in crop yields and agricultural sustainability. The article highlights the collaborative nature of the research, involving various experts and institutions, and emphasizes the project's potential impact on addressing global food supply challenges

In [62]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [63]:
memory_chain = LLMChain(llm=llm_chat, prompt=prompt_template_name, memory=memory)

In [66]:
query_1 = "What is the latest research in fighting drug-resistant infections?"

In [67]:
context_1 = retriever._get_relevant_documents(query_1)[0].page_content

In [68]:
context_1

"['Using an artificial intelligence algorithm, researchers at MIT and McMaster University have identified a new antibiotic that can kill a type of bacteria that is responsible for many drug-resistant infections.', '', 'If developed for use in patients, the drug could help to combat Acinetobacter baumannii, a species of bacteria that is often found in hospitals and can lead to pneumonia, meningitis, and other serious infections. The microbe is also a leading cause of infections in wounded soldiers in Iraq and Afghanistan.', '“Acinetobacter can survive on hospital doorknobs and equipment for long periods of time, and it can take up antibiotic resistance genes from its environment. It’s really common now to find A. baumannii isolates that are resistant to nearly every antibiotic,” says Jonathan Stokes, a former MIT postdoc who is now an assistant professor of biochemistry and biomedical sciences at McMaster University.', 'The researchers identified the new drug from a library of nearly 7,

In [70]:
formatted_prompt_template = prompt_template_name.format(
    context=context_1,
    question=query_1
)

In [74]:
# print(memory_chain.invoke({"context": context_1, "question": query_1}))

In [ ]:
print(chain.memory)

None


In [79]:
prompt_template_latest = """You are an assistant for answering questions. Use the following article content to answer the question.

    Article Content: {context}
    Question: {question}
    Answer:
"""

In [80]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

ANSWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=prompt_template_latest
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [83]:
# chain = prompt | llm_chat

# ai_msg = chain.invoke(
#     {
#         "messages": [
#             HumanMessage(
#                 content="Translate from English to French: I love programming."
#             ),
#             AIMessage(content="J'adore la programmation."),
#             HumanMessage(content="What did you just say?"),
#         ],
#     }
# )
# print(ai_msg.content)

In [89]:
# # Define chain
# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | ANSWER_PROMPT
#     | llm_chat  # Replace with your LLM instance (e.g., OpenAI or similar)
# )

In [91]:
query

'What does the article talk about J-WAFS Grand Challenge?'

In [92]:
from langchain.schema.runnable import RunnableMap
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document
from operator import itemgetter

In [93]:
query

'What does the article talk about J-WAFS Grand Challenge?'

NameError: name 'documents' is not defined

In [100]:
ANSWER_PROMPT = ChatPromptTemplate.from_template(
    """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible.
Each passage has a SOURCE which is the title of the document. When answering, cite source name of the passages you are answering from below the answer, on a new line, with a prefix of "SOURCE:".

context:
{context}

Question: {question}
Answer:
"""
)

DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {heading}
{page_content}
---
"""
)

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(
    """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
"""
)

standalone_question = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | llm_chat
    | StrOutputParser(),
)


def _format_chat_history(chat_history) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer


def _combine_documents(
    docs, document_prompt=DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


retrieved_documents = RunnableMap(
    docs=itemgetter("standalone_question") | retriever,
    question=itemgetter("standalone_question"),
)

_context = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": lambda x: x["question"],
}

answer = {
    "answer": _context | ANSWER_PROMPT | llm_chat,
    "docs": itemgetter("docs"),
}

chain = standalone_question | retrieved_documents | answer

ans = chain.invoke(
    {
        "question": "Who were awarded the intial grant for the J-WAFS Grand Challenge?",
        "chat_history": [
            query,
        ],
    }
)

print("---- Answer ----")
print(ans["answer"])

---- Answer ----
content='The initial grant for the J-WAFS Grand Challenge was awarded to a team of researchers led by Professor Matt Shoulders and research scientist Robert Wilson from the Department of Chemistry at MIT. Their proposal focuses on improving the efficiency of photosynthesis by enhancing the RuBisCO enzyme, which is crucial for converting carbon dioxide into energy-rich molecules in plants. The team will receive $1.5 million over three years for their multistage research project, which aims to tackle significant challenges in crop biology and agricultural productivity.\n\nSOURCE: Inaugural J-WAFS Grand Challenge aims to develop enhanced crop variants and move them from lab to land' additional_kwargs={} response_metadata={'token_usage': <OpenAIObject at 0x7eff041f53a0> JSON: {
  "prompt_tokens": 2521,
  "completion_tokens": 126,
  "total_tokens": 2647,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "rea

## Conversational QA

In [ ]:
from langchain.schema.runnable import RunnableMap
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document
from operator import itemgetter

In [ ]:
ANSWER_PROMPT = ChatPromptTemplate.from_template(
"""
You are an assistant for answering questions. Use the following article content to answer the question.

Article Content: {context}
Question: "{question}"
Answer:
"""
)

In [ ]:
DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)

In [ ]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(
    """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
"""
)

In [ ]:
def _format_chat_history(chat_history) -> str:
  buffer = ""
  for dialogue_turn in chat_history:
      human = "Human: " + dialogue_turn[0]
      ai = "Assistant: " + dialogue_turn[1]
      buffer += "\n" + "\n".join([human, ai])
  return buffer

In [ ]:
def _combine_documents(
    docs, document_prompt, document_separator="\n\n"
) -> str:
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [ ]:
retrieved_documents = RunnableMap(
    docs = itemgetter("chat_history"),
    context = lambda docs: _combine_documents(docs, ANSWER_PROMPT)
)

_context = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": lambda x: x["question"],
}


## Vertex AI Approach

In [26]:
vector_store_1 = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name=elastic_index_name,
    embedding=embeddings,
)

In [27]:
query = "Common, some way of development must be there!!!"
results = vector_store_1.similarity_search(query=query, k=5)

In [28]:
results

[Document(metadata={'date': '2021-11-02T00:00:00', 'author': 'Terri Park', 'source': 'MIT Schwarzman College of Computing', 'heading': '3 Questions: Blending computing with other disciplines at MIT', 'description': 'The Common Ground for Computing Education is facilitating collaborations to develop new classes for students to pursue computational knowledge within the context of their fields of interest.'}, page_content="The demand for computing-related training is at an all-time high. At MIT, there has been a remarkable tide of interest in computer science programs, with heavy enrollment from students studying everything from economics to life sciences eager to learn how computational techniques and methodologies can be used and applied within their primary field.', 'Launched in 2020, the Common Ground for Computing Education was created through the MIT Stephen A. Schwarzman College of Computing to meet the growing need for enhanced curricula that connect computer science and artificia